In [84]:
import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [85]:
# Charger les données et le modèle
team_dict = pd.read_csv('../team_dict.csv')
model = joblib.load('../Random_forest.sav')
match = pd.read_csv('dataset_test.csv')

# Dictionnaire des IDs d'équipe
team_ids = team_dict.set_index('Team')['ID'].to_dict()
id_to_team = {v: k for k, v in team_ids.items()}

In [86]:
def preprocess_data(df, labelencoded=False):
    # Convertir les noms d'équipe en IDs en utilisant le dictionnaire team_ids
    df['HomeTeam'] = df['HomeTeam'].replace(team_ids).astype('int32')
    df['AwayTeam'] = df['AwayTeam'].replace(team_ids).astype('int32')

    if not labelencoded:
        le = LabelEncoder()
        le.classes_ = np.array(['NH', 'H'])
        df['FTR'] = le.transform(df['FTR'])
    return df

# Prétraiter les données
datatest = preprocess_data(match, labelencoded=True)

/var/folders/40/rhs8rzb90455pxb5mrry2gnm0000gn/T/ipykernel_71548/3863793442.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HomeTeam'] = df['HomeTeam'].replace(team_ids).astype('int32')
/var/folders/40/rhs8rzb90455pxb5mrry2gnm0000gn/T/ipykernel_71548/3863793442.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['AwayTeam'] = df['AwayTeam'].replace(team_ids).astype('int32')


In [87]:
# Sélectionner les colonnes de caractéristiques
features = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC',
            'ATGC', 'HTP', 'ATP', 'MW', 'HTFormPts', 'ATFormPts',
            'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
            'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
            'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts']

In [88]:
# Extraire les caractéristiques pour les tests
X_test = match[features]

# prédictions
predicted_results = model.predict(X_test)

In [89]:
# Assurez-vous que la colonne 'FTR' est de type str
match['FTR'] = match['FTR'].astype(str)

In [90]:
# Afficher le nom des équipes prédites et le résultat de la prédiction
for i, (home_team_id, away_team_id, prediction) in enumerate(zip(X_test['HomeTeam'], X_test['AwayTeam'], predicted_results)):
    home_team_name = id_to_team[home_team_id]
    away_team_name = id_to_team[away_team_id]
    
    # Traduire la prédiction en résultat de match
    prediction_result = 'NH' if prediction == 0 else 'H'
    
    # Mettre à jour la colonne FTR dans le DataFrame match
    match.at[i, 'FTR'] = prediction_result
    
    # Remplacer les IDs par les noms des équipes
    match.at[i, 'HomeTeam'] = home_team_name
    match.at[i, 'AwayTeam'] = away_team_name
    
    # Afficher le nom des équipes prédites et le résultat de la prédiction
    print(f"Match: {home_team_name} vs {away_team_name}, Prediction: {prediction_result}")

# Sauvegarder les résultats dans un fichier CSV
match.to_csv('dataset_test_with_predictions.csv', index=False)
print("Les prédictions ont été sauvegardées dans 'dataset_test_with_predictions.csv'")

Match: Charlton vs Man City, Prediction: H
Match: Man City vs Sunderland, Prediction: H
Match: Man United vs Bradford, Prediction: H
Match: Middlesbrough vs Everton, Prediction: NH
Match: Coventry vs West Ham, Prediction: NH
Match: West Ham vs Bradford, Prediction: NH
Match: Charlton vs Middlesbrough, Prediction: H
Match: Sunderland vs Coventry, Prediction: H
Match: Aston Villa vs Tottenham, Prediction: H
Match: Newcastle vs Sunderland, Prediction: NH
Match: Aston Villa vs Newcastle, Prediction: NH
Match: Sunderland vs Middlesbrough, Prediction: H
Match: Charlton vs Everton, Prediction: H
Match: Middlesbrough vs Liverpool, Prediction: H
Match: Leicester vs Bradford, Prediction: NH
Match: Chelsea vs Ipswich, Prediction: H
Match: Leeds vs Coventry, Prediction: H
Match: Arsenal vs Ipswich, Prediction: H
Match: Middlesbrough vs Southampton, Prediction: NH
Match: Aston Villa vs Ipswich, Prediction: H
Match: Liverpool vs Man United, Prediction: H
Match: Tottenham vs Bradford, Prediction: H
M

/var/folders/40/rhs8rzb90455pxb5mrry2gnm0000gn/T/ipykernel_71548/200396049.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Charlton' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  match.at[i, 'HomeTeam'] = home_team_name
/var/folders/40/rhs8rzb90455pxb5mrry2gnm0000gn/T/ipykernel_71548/200396049.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Man City' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  match.at[i, 'AwayTeam'] = away_team_name
